<a href="https://colab.research.google.com/github/ayucd/HTML-Generator/blob/main/HTML_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and loading the dependencies

In [1]:
!pip install -q -U trl torch transformers git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install sentencepiece
!pip install python-dotenv
!pip install huggingface_hub
!pip install evaluate
!pip install accelerate
!pip install safetensors
!pip install ipywidgets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer


# Setup device-agnostic code



In [2]:
# Setup device-agnostic code
# Uncomment this

import torch
if torch.cuda.is_available():
    device = "cuda" # NVIDIA GPU
elif torch.backends.mps.is_built():
    device = "mps" # Apple GPU
else:
    device = "cpu"

print(f"Using device: {device}")

Using device: cuda


In [3]:
import os
os.environ['MAX_SPLIT_SIZE_GB'] = '32'
#device = torch.device("cuda")  # Make sure you're using the correct device
mem_info = torch.cuda.memory_stats(device=device)
memory_usage_bytes = mem_info.get("allocated_bytes.all.current")
torch.cuda.empty_cache()

# Set the global parameters for ease of use

In [5]:
# The model that we want to train from the Hugging Face hub
model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
# The instruction dataset to use
dataset_name = "ttbui/html_alpaca"
# Dataset split
dataset_split= "train"
# Fine-tuned model name
new_model = "llama-2-7b-html-generator-ac"
# Huggingface repository
hf_model_repo="sparklemidi/"+new_model
# Load the entire model on the GPU 0
device_map = {"": 0}

################################################################################
# bitsandbytes parameters
################################################################################
# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_double_nested_quant = False

################################################################################
# QLoRA parameters
################################################################################
# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = new_model
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1 # 2
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4 #1e-5
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine" #"constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = False
# Save checkpoint every X updates steps
save_steps = 0
# Log every X updates steps
logging_steps = 25
# Disable tqdm
disable_tqdm= True

################################################################################
# SFTTrainer parameters
################################################################################
# Maximum sequence length to use
max_seq_length = 2048 #None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True #False

# Connect to Hugging Face Hub using WRITE token

You can log in to Hugging Face Hub interactively. For this Go to Hugging Face -> Settings -> Access Tokens -> New Token and create a new token with access to WRITE.

In [6]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

# Load the dataset

In [7]:
# Load dataset from the hub
dataset = load_dataset(dataset_name, split=dataset_split)
# remove "response" column
dataset = dataset.remove_columns("response")
# for this project we only take the first 500 rows
dataset = dataset.select(range(500))
# Show dataset size
print(f"dataset size: {len(dataset)}")
# Show an example
print(dataset[randrange(len(dataset))])

Generating train split: 0 examples [00:00, ? examples/s]

dataset size: 200
{'output': '<!DOCTYPE html>\n<html>\n<head>\n  <title>Survey Form</title>\n</head>\n<body>\n  <form action="/action_page.php">\n    <label for="name">Name:</label><br>\n    <input type="text" id="name" name="name" placeholder="Your name.." required><br>\n    <label for="email">Email:</label><br>\n    <input type="text" id="email" name="email" placeholder="Your email.." required><br>\n    <label for="age">Age:</label><br>\n    <input type="number" id="age" name="age" placeholder="Your age.." required><br><br>\n    <fieldset>\n      <legend>Gender:</legend>\n      <input type="radio" id="male" name="gender" value="male" checked>\n      <label for="male">Male</label><br>\n      <input type="radio" id="female" name="gender" value="female">\n      <label for="female">Female</label><br>\n      <input type="radio" id="other" name="gender" value="other">\n      <label for="other">Other</label><br>\n    </fieldset>\n    <fieldset>\n      <legend>Interests:</legend>\n      <inp

In [8]:
# Check the dataset structure
dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 200
})

In [9]:
# Show a random example
print(dataset[randrange(len(dataset))])

{'output': '<!DOCTYPE html>\n<html lang="en">\n<head>\n <meta charset="UTF-8">\n <meta name="viewport" content="width=device-width, initial-scale=1.0">\n <title>Document</title>\n</head>\n<body>\n <header>\n   <nav>\n     <a href="#">Link 1</a>\n     <a href="#">Link 2</a>\n   </nav>\n </header>\n\n </body>\n</html>', 'input': '', 'instruction': 'Write a basic HTML structure with two navigation links'}


To fine-tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our instruction format.

In [10]:
# Set the instruction format
def format_instruction(sample):
	return f"""### Instruction:
Below is an instruction that describes a task, paired with an input that provides further context. Write a code that appropriately completes the request.

### Task:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}
"""

In [11]:
# Show a formatted instruction
print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Below is an instruction that describes a task, paired with an input that provides further context. Write a code that appropriately completes the request.

### Task:
Write an HTML page that displays a table containing the following columns: "Name", "Age", and "Marital Status".

### Input:


### Response:
<html>
<head>
  <title>Example Table</title>
</head>
<body>
  <table>
    <tr>
      <th>Name</th>
      <th>Age</th>
      <th>Marital Status</th>
    </tr>
    <tr>
      <td>John</td>
      <td>25</td>
      <td>Single</td>
    </tr>
    <tr>
      <td>Jane</td>
      <td>30</td>
      <td>Married</td>
    </tr>
  </table>
</body>
</html>



# Instruction Fine-tuning a sharded LLaMA-2 7B model using SFTTrainer

We will use the recently introduced method in the paper "QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation" by Tim Dettmers et al. QLoRA is a new technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance.

Quantize the pre-trained model to 4 bits and freeze it.
Attach small, trainable adapter layers. (LoRA)
Finetune only the adapter layers while using the frozen quantized model for context.

In [12]:
# Get the type
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_double_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

In [13]:
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache = False, device_map=device_map)
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [14]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
)

In [15]:
# Define the training arguments
args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size, # 6 if use_flash_attention else 4,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
    #save_steps=save_steps,
    logging_steps=logging_steps,
    save_strategy="epoch",
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    #max_steps=max_steps,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    disable_tqdm=disable_tqdm,
    report_to="tensorboard",
    seed=42
)

In [16]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=packing,
    formatting_func=format_instruction,
    args=args,
)

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model in local
trainer.save_model()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


OutOfMemoryError: ignored

# Merge the model and the adapters and save it

When running in a T4 instance we have to clean the memory.

In [18]:
# Empty VRAM when using T4
del model
del trainer
import gc
gc.collect()
gc.collect()

0

In [19]:
torch.cuda.empty_cache() # PyTorch thing

In [20]:
gc.collect()

56

Reload the trained and saved model and merge it then we can save the whole model

In [ ]:
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# Merge LoRA and base model
merged_model = new_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

In [ ]:
# push merged model to the hub
merged_model.push_to_hub(hf_model_repo)
tokenizer.push_to_hub(hf_model_repo)

# Test the merged model

It is time to check our model performance

In [ ]:
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Below is an instruction that describes a task, paired with an input that provides further context. Write the programming code that appropriately completes the request.

### Task:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"\nGround truth:\n{sample['output']}")

# Load the model from the Hub to test it

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)
# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, load_in_4bit=True, torch_dtype=torch.float16,
                                             device_map=device_map)
# Create an instruction
instruction="Write the HTML code for a basic About page."
input=""

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""
# Tokenize the input
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# Run the model to infere an output
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

# Print the result
print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")